In [1]:
#import basic python libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.6 MB/s eta 0:00:00


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
input_file = pd.read_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Recipe_cfp.csv', encoding='latin1')
input_file

,ID,ingredients,region
0,709824,"['black pepper', 'cumin', 'garlic', 'sea salt'...",Middle Eastern
1,905901,"['green bean carrot', 'vegetable oil', 'bay le...",Middle Eastern
2,725971,"['okra', 'salt black pepper', 'lamb shoulder',...",Middle Eastern
3,114702,"['cornmeal', 'cumin', 'sauce', 'garlic', 'cori...",Middle Eastern
4,440574,"['black pepper', 'sea salt', 'cumin seed', 'co...",Middle Eastern
...,...,...,...
118078,523830,"['yellow pepper', 'mushroom', 'garlic', 'jalap...",Canadian
118079,660518,"['cardamom seed', 'asian pear', 'water', 'ging...",Canadian
118080,742230,"['potato', 'pepper', 'vegetable stock', 'salt'...",Canadian
118081,779223,"['baking powder', 'vanilla', 'flour', 'sugar',...",Canadian


In [7]:
input_file2 = pd.read_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Recipe_ingredient_cfp.csv', encoding='latin1')
input_file2

,recipe_no,ingredient
0,709824,water
1,709824,red lentil
2,709824,rom tomato
3,709824,carrot
4,709824,onion
...,...,...
1154399,674570,date
1154400,674570,butter
1154401,674570,vanilla
1154402,674570,cinnamon


In [8]:
rdb_ing = list(input_file2['ingredient'].unique())
# rdb_ing = rdb_ing.upper()
# print(rdb_ing)
print(len(rdb_ing))

20280


In [9]:
input_file_fooditem = pd.read_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Food_Items_CF_clean_1.csv', encoding='latin1')

Exact match

In [10]:
mapped_food_items = input_file_fooditem['FOOD COMMODITY ITEM updated'].str.upper().tolist()
mapped_food_typology = input_file_fooditem['FOOD COMMODITY TYPOLOGY updated'].str.upper().tolist()

# mapped count
mapped_total = 0
mapped_item = 0
mapped_typo = 0

# mapped ingredients
mapped_item_ing =[]
mapped_typo_ing =[]
mapped_total_ing =[]

for ingredient in rdb_ing:
    ingredient = ingredient.upper()
    if ingredient in mapped_food_items:
        mapped_item_ing.append(ingredient)
        mapped_item += 1

    if ingredient in mapped_food_typology:
        mapped_typo += 1
        mapped_typo_ing.append(ingredient)

    if ingredient in mapped_food_items or ingredient in mapped_food_typology:
        mapped_total += 1
        mapped_total_ing.append(ingredient)

print("Number of Sueatable food items mapped with RecipeDB:", mapped_item)
print("Number of Sueatable food typologies mapped with RecipeDB:", mapped_typo)
print("Number of Sueatable food products mapped with RecipeDB:", mapped_total)

# print("Sueatable food items mapped with RecipeDB:", mapped_item_ing)
# print("Sueatable food typo mapped with RecipeDB:", mapped_typo_ing)
# print("Sueatable food products mapped with RecipeDB:", mapped_total_ing)


Number of Sueatable food items mapped with RecipeDB: 178
Number of Sueatable food typologies mapped with RecipeDB: 44
Number of Sueatable food products mapped with RecipeDB: 207


In [ ]:
# this above discrepency is due to duplicate entries present i.e. butter is present in food item and food typo both.

In [11]:
import pandas as pd

words = set(input_file_fooditem['FOOD COMMODITY ITEM updated'].str.upper())
words.update(input_file_fooditem['FOOD COMMODITY TYPOLOGY updated'].str.upper())

recipe_match = []

for ingredient in rdb_ing:
    ingredient_upper = ingredient.upper()

    if ingredient_upper in words:
        matching_ingredient = input_file_fooditem[(input_file_fooditem['FOOD COMMODITY ITEM updated'].str.upper() == ingredient_upper) |
                                                 (input_file_fooditem['FOOD COMMODITY TYPOLOGY updated'].str.upper() == ingredient_upper)]

        if not matching_ingredient.empty:
            matched_ingredient = matching_ingredient['FOOD COMMODITY ITEM updated'].iloc[0]
            recipe_match_dict = {
                'Ingredient': ingredient,
                'Mapped_Ingredient': matched_ingredient
            }
            recipe_match.append(recipe_match_dict)

# Create a DataFrame from the list of dictionaries
df_match_exact = pd.DataFrame(recipe_match)
df_match_exact = df_match_exact.drop_duplicates(subset=['Ingredient'])
df_match_exact


,Ingredient,Mapped_Ingredient
0,water,WATER
1,carrot,CARROT
2,onion,ONION
3,garlic,GARLIC
4,olive oil,OLIVE OIL
...,...,...
202,grana padano,GRANA PADANO
203,wine red,WINE RED
204,egg pasta,EGG PASTA
205,rye,RYE


# RecipeDB Ingredients not matched directly

In [14]:
# Convert the lists to sets
set1 = set(rdb_ing)
set2 = {ingredient.lower() for ingredient in mapped_total_ing}

# Find the different ingredients
left_unmapped_em  = list(set1 - set2)

print(len(left_unmapped_em))

20073


# Map both dataset based on similarity using BERT embeddings: BERT-BASE-UNCASED

In [16]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
model_name = 'bert-base-uncased'  # BERT model variant
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [18]:
ingredients1 = ["wheat plain flour", "whole wheat flour", "beef stock dark"]
ingredients2 = ["wheat flour", "beef bone free", "beef with bone"]

# Tokenize and convert to BERT input tensors
inputs1 = tokenizer(ingredients1, padding=True, truncation=True, return_tensors="pt")
inputs2 = tokenizer(ingredients2, padding=True, truncation=True, return_tensors="pt")

# Obtain BERT embeddings
with torch.no_grad():
    embeddings1 = model(**inputs1).last_hidden_state[:, 0, :]  # Take the CLS token embedding
    embeddings2 = model(**inputs2).last_hidden_state[:, 0, :]


In [19]:
similarity_matrix = cosine_similarity(embeddings1, embeddings2)
similarity_matrix

array([[0.9730221 , 0.8066001 , 0.78121275],
       [0.96798563, 0.7410127 , 0.6987342 ],
       [0.8581802 , 0.9053152 , 0.8863512 ]], dtype=float32)

# Mapping Sueatable to RecipeDB

In [22]:
ingredient_counts = input_file2['ingredient'].value_counts()

ingredient_dict = ingredient_counts.to_dict()

recipedb_ing = pd.DataFrame({'Ingredient': ingredient_counts.index, 'Frequency': ingredient_counts.values})
recipedb_ing

,Ingredient,Frequency
0,salt,45206
1,onion,34548
2,butter,27013
3,garlic clove,24893
4,water,24773
...,...,...
20275,milk chocolate hershey,1
20276,chocolate flavor crisp rice cereal,1
20277,chocolate ripple ice cream,1
20278,caramel topping syrup,1


# MApping using Jaccard Similarity

In [23]:
import pandas as pd
from nltk.tokenize import word_tokenize

ingredients1 = recipedb_ing['Ingredient']

ingredients2 = input_file_fooditem['FOOD COMMODITY ITEM updated']

jaccard = pd.DataFrame(columns=["RecipeDB ing", "Sueatable ing", "Similarity"])

# Tokenize the ingredients
tokens1 = [set(word_tokenize(ingredient.lower())) for ingredient in ingredients1]
tokens2 = [set(word_tokenize(ingredient.lower())) for ingredient in ingredients2]

# Calculate Jaccard similarity and add to the DataFrame
for i in range(len(ingredients1)):
    for j in range(len(ingredients2)):
        intersection = len(tokens1[i].intersection(tokens2[j]))
        union = len(tokens1[i].union(tokens2[j]))
        jaccard_similarity = intersection / union

        if jaccard_similarity > 0:
            jaccard = jaccard.append({"RecipeDB ing": ingredients1[i], "Sueatable ing": ingredients2[j], "Similarity": jaccard_similarity},
                           ignore_index=True)

jaccard


,RecipeDB ing,Sueatable ing,Similarity
0,onion,ONION,1.00
1,butter,PEANUT BUTTER,0.50
2,butter,BUTTER,1.00
3,garlic clove,PESTO WITHOUT GARLIC,0.25
4,garlic clove,GARLIC,0.50
...,...,...,...
53399,chocolate ripple ice cream,ALMOND CHOCOLATE,0.20
53400,chocolate ripple ice cream,HAZELNUT CHOCOLATE,0.20
53401,chocolate ripple ice cream,HAZELNUT CREAM,0.20
53402,chocolate ripple ice cream,SOY CREAM,0.20


In [24]:
# Create bins of similarity values
bins = pd.cut(jaccard['Similarity'], bins=[0, 0.2, 0.4, 0.6, 0.8, 1.0])
bin_counts = bins.value_counts().sort_index()

print(bin_counts)

(0.0, 0.2]    16628
(0.2, 0.4]    32754
(0.4, 0.6]     3367
(0.6, 0.8]      457
(0.8, 1.0]      198
Name: Similarity, dtype: int64


# Mapping using BERT ST: bert-base-nli-mean-tokens

In [25]:
!pip install --quiet sentence-transformers

In [26]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [27]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

ingredients1 = recipedb_ing['Ingredient']
ingredients2 = input_file_fooditem['FOOD COMMODITY ITEM updated']
ingredients2_lower = [ingredient.lower() for ingredient in ingredients2]

embeddings1 = model.encode(ingredients1)
embeddings2 = model.encode(ingredients2_lower)


In [28]:
similarity_matrix_st_bert = cosine_similarity(embeddings1, embeddings2)
similarity_matrix_st_bert


array([[0.40754807, 0.40657878, 0.451334  , ..., 0.4662223 , 0.49814472,
        0.46529883],
       [0.30862892, 0.27916735, 0.3790871 , ..., 0.49731296, 0.49173272,
        0.38902748],
       [0.357436  , 0.3297541 , 0.41928136, ..., 0.36371428, 0.38874754,
        0.4003945 ],
       ...,
       [0.32885987, 0.35297692, 0.39836267, ..., 0.26449803, 0.3070593 ,
        0.3061311 ],
       [0.42789206, 0.4371999 , 0.45439434, ..., 0.19169395, 0.26686892,
        0.22917336],
       [0.29919916, 0.31331098, 0.35530058, ..., 0.26275796, 0.3501966 ,
        0.26082587]], dtype=float32)

In [29]:
# Convert the similarity matrix into a DataFrame
ingredient_mapping = pd.DataFrame(similarity_matrix_st_bert, index=ingredients1, columns=ingredients2_lower)

ingredient_mapping

,beer in can,beer in glass,beer modular can,chocolate cream cooky,cooky,water,bread multicereal,bread plain,bread whole,bread frozen,...,lobster,mussel,cuttlefish,octopus,squid,prawn,shrimp,octopus frozen,shrimp frozen,prawn frozen
Ingredient,,,,,,,,,,,,,,,,,,,,,
salt,0.407548,0.406579,0.451334,0.389121,0.596639,0.670299,0.613382,0.597211,0.622559,0.530091,...,0.540180,0.717019,0.552751,0.571068,0.576672,0.598963,0.598379,0.466222,0.498145,0.465299
onion,0.308629,0.279167,0.379087,0.450285,0.710122,0.493120,0.613386,0.527409,0.604342,0.463710,...,0.699483,0.630287,0.505892,0.650432,0.571044,0.566215,0.639481,0.497313,0.491733,0.389027
butter,0.357436,0.329754,0.419281,0.661703,0.716410,0.524945,0.694195,0.596598,0.726739,0.499978,...,0.513322,0.538317,0.478393,0.452700,0.463968,0.535897,0.475704,0.363714,0.388748,0.400394
garlic clove,0.270338,0.250571,0.292180,0.551835,0.552685,0.289317,0.578709,0.393496,0.543114,0.362563,...,0.457832,0.453847,0.435324,0.386509,0.350557,0.380384,0.447528,0.267849,0.324654,0.248155
water,0.439130,0.447892,0.461253,0.166349,0.497104,1.000000,0.415187,0.410627,0.416243,0.387941,...,0.388643,0.756920,0.432181,0.594909,0.645414,0.650948,0.505302,0.458512,0.400623,0.477783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
milk chocolate hershey,0.428363,0.416913,0.483837,0.857660,0.260720,0.142113,0.467911,0.372334,0.470361,0.378669,...,0.241149,0.231279,0.242838,0.159520,0.119567,0.276318,0.224100,0.253546,0.315370,0.337902
chocolate flavor crisp rice cereal,0.266659,0.287515,0.327797,0.818652,0.239144,0.027001,0.447259,0.409444,0.463956,0.234810,...,0.208889,0.153721,0.189937,0.108191,0.068413,0.171469,0.209853,0.122465,0.196587,0.146544
chocolate ripple ice cream,0.328860,0.352977,0.398363,0.925916,0.307432,0.181870,0.482477,0.355360,0.480506,0.350942,...,0.266954,0.235681,0.226330,0.188867,0.159586,0.256528,0.239888,0.264498,0.307059,0.306131


In [30]:
max_similarities = np.max(similarity_matrix_st_bert, axis=1)
max_indices = np.argmax(similarity_matrix_st_bert, axis=1)

df = pd.DataFrame(columns=["RecipeDB Ingredient", "Sueatable Ingredient", "Similarity"])

for i in range(len(ingredients1)):
    ingredient1 = ingredients1[i]
    max_similarity = max_similarities[i]
    max_index = max_indices[i]
    most_similar_ingredient = ingredients2[max_index]
    df = df.append({"RecipeDB Ingredient": ingredient1, "Sueatable Ingredient": most_similar_ingredient, "Similarity": max_similarity}, ignore_index=True)

df

,RecipeDB Ingredient,Sueatable Ingredient,Similarity
0,salt,PEPPER,0.771985
1,onion,ONION,1.000000
2,butter,BUTTER,1.000000
3,garlic clove,GARLIC,0.930919
4,water,WATER,1.000000
...,...,...,...
20275,milk chocolate hershey,MILK CHOCOLATE,0.947778
20276,chocolate flavor crisp rice cereal,CHOCOLATE CREAM COOKY,0.818652
20277,chocolate ripple ice cream,CHOCOLATE CREAM COOKY,0.925916
20278,caramel topping syrup,CRANBERRY JUICE,0.852054


In [31]:
df.to_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Mapped/bert_nli_recipedb_sueat_mapping.csv', index=False)

In [32]:
df_bert = pd.read_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Mapped/bert_nli_recipedb_sueat_mapping.csv')

In [33]:
filtered_df = df_bert[(df_bert['Similarity'] > 0.81)]
filtered_df

,RecipeDB Ingredient,Sueatable Ingredient,Similarity
1,onion,ONION,1.000001
2,butter,BUTTER,1.000000
3,garlic clove,GARLIC,0.930919
4,water,WATER,1.000000
5,olive oil,OLIVE OIL,1.000000
...,...,...,...
20274,dutch cocoa hershey,HAZELNUT CHOCOLATE,0.823729
20275,milk chocolate hershey,MILK CHOCOLATE,0.947778
20276,chocolate flavor crisp rice cereal,CHOCOLATE CREAM COOKY,0.818652
20277,chocolate ripple ice cream,CHOCOLATE CREAM COOKY,0.925916


In [34]:
filtered_df.to_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/exp/80_81_map_bert.csv')

In [ ]:
filtered_df = pd.read_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/exp/80_81_map_bert.csv')

# Mapping using RoBERTa

In [35]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('roberta-base-nli-mean-tokens')

ingredients1 = recipedb_ing['Ingredient']
ingredients2 = input_file_fooditem['FOOD COMMODITY ITEM updated']
ingredients2_lower = [ingredient.lower() for ingredient in ingredients2]

embeddings1 = model.encode(ingredients1)
embeddings2 = model.encode(ingredients2_lower)

similarity_matrix_st_roberta = cosine_similarity(embeddings1, embeddings2)
similarity_matrix_st_roberta


array([[0.5113533 , 0.54409176, 0.56873286, ..., 0.3766689 , 0.32203496,
        0.4597208 ],
       [0.4607181 , 0.421774  , 0.4763264 , ..., 0.31114984, 0.21091664,
        0.3973515 ],
       [0.5305706 , 0.5050204 , 0.5674228 , ..., 0.31778747, 0.27295402,
        0.40148062],
       ...,
       [0.43007612, 0.4674998 , 0.52676076, ..., 0.5799695 , 0.5692391 ,
        0.62226903],
       [0.54850054, 0.54887724, 0.61199164, ..., 0.559111  , 0.6446602 ,
        0.62345886],
       [0.4126402 , 0.38551012, 0.51483625, ..., 0.56797636, 0.5384694 ,
        0.4468661 ]], dtype=float32)

In [36]:
# Convert the similarity matrix into a DataFrame
ingredient_mapping_rob = pd.DataFrame(similarity_matrix_st_roberta, index=ingredients1, columns=ingredients2_lower)

ingredient_mapping_rob

,beer in can,beer in glass,beer modular can,chocolate cream cooky,cooky,water,bread multicereal,bread plain,bread whole,bread frozen,...,lobster,mussel,cuttlefish,octopus,squid,prawn,shrimp,octopus frozen,shrimp frozen,prawn frozen
Ingredient,,,,,,,,,,,,,,,,,,,,,
salt,0.511353,0.544092,0.568733,0.330349,0.503838,0.628763,0.543537,0.543139,0.510955,0.469653,...,0.712795,0.703218,0.525219,0.490609,0.778854,0.626155,0.457762,0.376669,0.322035,0.459721
onion,0.460718,0.421774,0.476326,0.241365,0.664203,0.701855,0.567920,0.436381,0.568514,0.372006,...,0.730078,0.754130,0.453866,0.528949,0.759173,0.654903,0.435402,0.311150,0.210917,0.397352
butter,0.530571,0.505020,0.567423,0.305515,0.635836,0.690035,0.588853,0.517415,0.566863,0.413607,...,0.761244,0.725482,0.528242,0.523264,0.794985,0.662405,0.505277,0.317787,0.272954,0.401481
garlic clove,0.569424,0.504421,0.620160,0.523650,0.603308,0.413558,0.598557,0.473204,0.477132,0.477237,...,0.610402,0.542302,0.683936,0.667273,0.395096,0.564813,0.577208,0.591101,0.514820,0.446031
water,0.509937,0.521193,0.508492,0.175006,0.533231,1.000000,0.488228,0.419872,0.469053,0.312733,...,0.669426,0.756351,0.566851,0.548995,0.642343,0.606924,0.550882,0.364524,0.319715,0.374274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
milk chocolate hershey,0.433020,0.443967,0.493176,0.834926,0.504423,0.195330,0.590040,0.456531,0.403461,0.572650,...,0.389720,0.360890,0.418449,0.440214,0.199697,0.440702,0.407140,0.559993,0.549198,0.553874
chocolate flavor crisp rice cereal,0.393116,0.383629,0.444959,0.854235,0.418057,0.093401,0.511320,0.443820,0.351960,0.537981,...,0.259333,0.149047,0.262104,0.282028,0.139991,0.374526,0.269727,0.420973,0.438680,0.496794
chocolate ripple ice cream,0.430076,0.467500,0.526761,0.909879,0.430047,0.147789,0.530578,0.404567,0.324216,0.631456,...,0.348717,0.257282,0.394628,0.399633,0.172219,0.440657,0.353063,0.579970,0.569239,0.622269


In [37]:
max_similarities = np.max(similarity_matrix_st_roberta, axis=1)
max_indices = np.argmax(similarity_matrix_st_roberta, axis=1)

df_rob = pd.DataFrame(columns=["RecipeDB_Ingredient", "Sueatable_Ingredient", "Similarity"])

for i in range(len(ingredients1)):
    ingredient1 = ingredients1[i]
    max_similarity = max_similarities[i]
    max_index = max_indices[i]
    most_similar_ingredient = ingredients2[max_index]
    df_rob = df_rob.append({"RecipeDB_Ingredient": ingredient1, "Sueatable_Ingredient": most_similar_ingredient, "Similarity": max_similarity}, ignore_index=True)

df_rob

,RecipeDB_Ingredient,Sueatable_Ingredient,Similarity
0,salt,SOLE,0.843593
1,onion,ONION,1.000000
2,butter,BUTTER,1.000000
3,garlic clove,GARLIC,0.822195
4,water,WATER,1.000000
...,...,...,...
20275,milk chocolate hershey,MILK CHOCOLATE,0.953825
20276,chocolate flavor crisp rice cereal,CHOCOLATE CREAM COOKY,0.854235
20277,chocolate ripple ice cream,CHOCOLATE CREAM COOKY,0.909879
20278,caramel topping syrup,CHOCOLATE CREAM COOKY,0.782797


In [38]:
df_rob.to_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Mapped/roberta_nli_recipedb_sueat_map.csv')

# Mapping using Word2Vec

In [39]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

ingredient1 = recipedb_ing['Ingredient']
ingredient2 = input_file_fooditem['FOOD COMMODITY ITEM updated']

# Tokenize the sentences
tokens1 = [sent.lower().split() for sent in ingredient1]
tokens2 = [sent.lower().split() for sent in ingredient2]

# Train the Word2Vec model
model = Word2Vec(sentences=tokens1 + tokens2, min_count=1)

# Calculate sentence similarities and create DataFrame
similarity_scores = []
similar_ingredients = []
for tokens_s1 in tokens1:
    s1_embedding = sum(model.wv[word] for word in tokens_s1) / len(tokens_s1)

    max_similarity = 0.0
    max_similar_ingredient = ""
    for tokens_s2 in tokens2:
        s2_embedding = sum(model.wv[word] for word in tokens_s2) / len(tokens_s2)

        # Calculate cosine similarity between sentence embeddings
        similarity = cosine_similarity([s1_embedding], [s2_embedding])[0][0]
        if similarity > max_similarity:
            max_similarity = similarity
            max_similar_ingredient = " ".join(tokens_s2)

    similarity_scores.append(max_similarity)
    similar_ingredients.append(max_similar_ingredient)

df_wv = pd.DataFrame({'RecipeDB_Ingredient': ingredient1, 'Sueatable_Ingredient': similar_ingredients, 'Similarity': similarity_scores})

df_wv


,RecipeDB_Ingredient,Sueatable_Ingredient,Similarity
0,salt,green bean frozen,0.998170
1,onion,onion,1.000000
2,butter,butter,1.000000
3,garlic clove,garlic,0.999563
4,water,water,1.000000
...,...,...,...
20275,milk chocolate hershey,milk chocolate,0.999918
20276,chocolate flavor crisp rice cereal,chocolate cream cooky,0.999592
20277,chocolate ripple ice cream,chocolate cream cooky,0.999845
20278,caramel topping syrup,yeast liquid,0.999312


In [40]:
df_wv.to_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/Mapped/word2vec_recipedb_sueat_map.csv')

# Best Model --- BERT

In [12]:
filtered_df = pd.read_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/exp/80_81_map_bert.csv')
filtered_df

,Unnamed: 0,RecipeDB Ingredient,Sueatable Ingredient,Similarity
0,1,onion,ONION,1.000001
1,2,butter,BUTTER,1.000000
2,3,garlic clove,GARLIC,0.930919
3,4,water,WATER,1.000000
4,5,olive oil,OLIVE OIL,1.000000
...,...,...,...,...
7748,20274,dutch cocoa hershey,HAZELNUT CHOCOLATE,0.823729
7749,20275,milk chocolate hershey,MILK CHOCOLATE,0.947778
7750,20276,chocolate flavor crisp rice cereal,CHOCOLATE CREAM COOKY,0.818652
7751,20277,chocolate ripple ice cream,CHOCOLATE CREAM COOKY,0.925916


In [13]:
# Create a DataFrame to store the results
df_stringmatch = pd.DataFrame(columns=['Recipe ID', 'Recipe Ingredient', 'Total Ingredient',
                                  'Available Ingredients', 'Available Count', 'Not Available Ingredients',
                                  'Not Available Count', 'Available Percentage'])

In [14]:
import pandas as pd
import ast

available_ingredients = []
not_available_ingredients = []

for i in range(input_file['ID'].nunique()):
    recipe_ingredients = ast.literal_eval(input_file['ingredients'][i])
    recipe_available_ingredients = []
    recipe_not_available_ingredients = []

    for ingredient in recipe_ingredients:
        if ingredient in filtered_df['RecipeDB Ingredient'].values:
            recipe_available_ingredients.append(ingredient)
        else:
            recipe_not_available_ingredients.append(ingredient)

    available_ingredients.append(recipe_available_ingredients)
    not_available_ingredients.append(recipe_not_available_ingredients)


# Populate the DataFrame with the results
for i, recipe_id in enumerate(input_file['ID'].unique()):
    available_count = len(available_ingredients[i])
    not_available_count = len(not_available_ingredients[i])
    total_ingredients = available_count + not_available_count
    percentage_available = (available_count / total_ingredients) * 100 if total_ingredients > 0 else 0

    df_stringmatch = df_stringmatch.append({
        'Recipe ID': recipe_id,
        'Recipe Ingredient': input_file['ingredients'][i],
        'Total Ingredient': len(input_file['ingredients'][i].split(',')),
        'Available Ingredients': available_ingredients[i],
        'Available Count': available_count,
        'Not Available Ingredients': not_available_ingredients[i],
        'Not Available Count': not_available_count,
        'Available Percentage': percentage_available
    }, ignore_index=True)

df_stringmatch


,Recipe ID,Recipe Ingredient,Total Ingredient,Available Ingredients,Available Count,Not Available Ingredients,Not Available Count,Available Percentage
0,709824,"['black pepper', 'cumin', 'garlic', 'sea salt'...",10,"[cumin, garlic, carrot, coriander, red lentil,...",8,"[black pepper, sea salt]",2,80.000000
1,905901,"['green bean carrot', 'vegetable oil', 'bay le...",9,"[green bean carrot, vegetable oil, garlic, oni...",6,"[bay leaf, cardamom, salt black pepper]",3,66.666667
2,725971,"['okra', 'salt black pepper', 'lamb shoulder',...",7,"[okra, lamb shoulder, tomato sauce, water, oni...",6,[salt black pepper],1,85.714286
3,114702,"['cornmeal', 'cumin', 'sauce', 'garlic', 'cori...",10,"[cornmeal, cumin, garlic, coriander, garbanzo ...",8,"[sauce, salt]",2,80.000000
4,440574,"['black pepper', 'sea salt', 'cumin seed', 'co...",6,"[coriander seed, sesame seed, hazelnut]",3,"[black pepper, sea salt, cumin seed]",3,50.000000
...,...,...,...,...,...,...,...,...
118078,523830,"['yellow pepper', 'mushroom', 'garlic', 'jalap...",10,"[mushroom, garlic, celery, red pepper, salt pe...",5,"[yellow pepper, jalapeno pepper, vegetable oil...",5,50.000000
118079,660518,"['cardamom seed', 'asian pear', 'water', 'ging...",4,"[asian pear, water, ginger]",3,[cardamom seed],1,75.000000
118080,742230,"['potato', 'pepper', 'vegetable stock', 'salt'...",6,"[potato, pepper, vegetable stock, onion, olive...",5,[salt],1,83.333333
118081,779223,"['baking powder', 'vanilla', 'flour', 'sugar',...",7,"[vanilla, flour, sugar, margarine, liquid]",5,"[baking powder, salt]",2,71.428571


In [15]:
df_stringmatch.to_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/FinalList_with_string_matching_mansi.csv', encoding='utf-8', index=False)

Recipes with exact mapping

In [16]:
df_match_exact

,Ingredient,Mapped_Ingredient
0,water,WATER
1,carrot,CARROT
2,onion,ONION
3,garlic,GARLIC
4,olive oil,OLIVE OIL
...,...,...
202,grana padano,GRANA PADANO
203,wine red,WINE RED
204,egg pasta,EGG PASTA
205,rye,RYE


In [17]:
import pandas as pd
import ast

available_ingredients = []
not_available_ingredients = []

for i in range(input_file['ID'].nunique()):
    recipe_ingredients = ast.literal_eval(input_file['ingredients'][i])
    recipe_available_ingredients = []
    recipe_not_available_ingredients = []

    for ingredient in recipe_ingredients:
        if ingredient in df_match_exact['Ingredient'].values:
            recipe_available_ingredients.append(ingredient)
        else:
            recipe_not_available_ingredients.append(ingredient)

    available_ingredients.append(recipe_available_ingredients)
    not_available_ingredients.append(recipe_not_available_ingredients)

df_exactmatch = pd.DataFrame(columns=['Recipe ID', 'Recipe Ingredient', 'Total Ingredient',
                                       'Available Ingredients', 'Available Count', 'Not Available Ingredients',
                                       'Not Available Count', 'Available Percentage'])

for i, recipe_id in enumerate(input_file['ID'].unique()):
    available_count = len(available_ingredients[i])
    not_available_count = len(not_available_ingredients[i])
    total_ingredients = available_count + not_available_count
    percentage_available = (available_count / total_ingredients) * 100 if total_ingredients > 0 else 0

    df_exactmatch = df_exactmatch.append({
        'Recipe ID': recipe_id,
        'Recipe Ingredient': input_file['ingredients'][i],
        'Total Ingredient': len(input_file['ingredients'][i].split(',')),
        'Available Ingredients': available_ingredients[i],
        'Available Count': available_count,
        'Not Available Ingredients': not_available_ingredients[i],
        'Not Available Count': not_available_count,
        'Available Percentage': percentage_available
    }, ignore_index=True)

df_exactmatch


,Recipe ID,Recipe Ingredient,Total Ingredient,Available Ingredients,Available Count,Not Available Ingredients,Not Available Count,Available Percentage
0,709824,"['black pepper', 'cumin', 'garlic', 'sea salt'...",10,"[garlic, carrot, water, onion]",4,"[black pepper, cumin, sea salt, coriander, red...",6,40.000000
1,905901,"['green bean carrot', 'vegetable oil', 'bay le...",9,"[garlic, onion]",2,"[green bean carrot, vegetable oil, bay leaf, c...",7,22.222222
2,725971,"['okra', 'salt black pepper', 'lamb shoulder',...",7,"[water, onion, olive oil]",3,"[okra, salt black pepper, lamb shoulder, tomat...",4,42.857143
3,114702,"['cornmeal', 'cumin', 'sauce', 'garlic', 'cori...",10,"[garlic, onion, cream, olive oil]",4,"[cornmeal, cumin, sauce, coriander, garbanzo b...",6,40.000000
4,440574,"['black pepper', 'sea salt', 'cumin seed', 'co...",6,"[sesame seed, hazelnut]",2,"[black pepper, sea salt, cumin seed, coriander...",4,33.333333
...,...,...,...,...,...,...,...,...
118078,523830,"['yellow pepper', 'mushroom', 'garlic', 'jalap...",10,"[mushroom, garlic, celery]",3,"[yellow pepper, jalapeno pepper, vegetable oil...",7,30.000000
118079,660518,"['cardamom seed', 'asian pear', 'water', 'ging...",4,"[water, ginger]",2,"[cardamom seed, asian pear]",2,50.000000
118080,742230,"['potato', 'pepper', 'vegetable stock', 'salt'...",6,"[potato, pepper, onion, olive oil]",4,"[vegetable stock, salt]",2,66.666667
118081,779223,"['baking powder', 'vanilla', 'flour', 'sugar',...",7,"[vanilla, sugar, margarine]",3,"[baking powder, flour, salt, liquid]",4,42.857143


In [18]:
df_exactmatch.to_csv('/content/drive/MyDrive/Carbon_footprint_Mansi/Data/FinalList_without_string_matching_mansi.csv', encoding='utf-8', index=False)